# 데이터 전처리

In [179]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import math
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

df = pd.read_csv('https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/datasets/vgames2.csv')
df.drop(['Unnamed: 0'], axis = 1, inplace = True)
print(df.dtypes)
print(df.isna().sum())
df.head(5)

Name            object
Platform        object
Year           float64
Genre           object
Publisher       object
NA_Sales        object
EU_Sales        object
JP_Sales        object
Other_Sales     object
dtype: object
Name             0
Platform         0
Year           271
Genre           50
Publisher       58
NA_Sales         0
EU_Sales         0
JP_Sales         0
Other_Sales      0
dtype: int64


,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales
0,Candace Kane's Candy Factory,DS,2008.0,Action,Destineer,0.04,0,0,0
1,The Munchables,Wii,2009.0,Action,Namco Bandai Games,0.17,0,0,0.01
2,Otome wa Oanesama Boku ni Koi Shiteru Portable,PSP,2010.0,Adventure,Alchemist,0,0,0.02,0
3,Deal or No Deal: Special Edition,DS,2010.0,Misc,Zoo Games,0.04,0,0,0
4,Ben 10 Ultimate Alien: Cosmic Destruction,PS3,2010.0,Platform,D3Publisher,0.12,0.09,0,0.04


In [180]:
def unitConversion(v): #기본 단위를 M으로 통일
  if 'K' in v:
    return float(v.replace('K','')) * 0.001

  return float(v.replace('M', ''))

sale_columns = df.filter(regex='_Sales').columns  #_Sales 포함한 컬럼 선택
for c in sale_columns:
  df[c] = df[c].apply(unitConversion)

df['Total_Sales'] = df[sale_columns].sum(axis=1)
df.dtypes

Name            object
Platform        object
Year           float64
Genre           object
Publisher       object
NA_Sales       float64
EU_Sales       float64
JP_Sales       float64
Other_Sales    float64
Total_Sales    float64
dtype: object

In [181]:
since1900 = (df['Year'] < 100) & (df['Year'] >= 30)
df[since1900].head(5)

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Total_Sales
81,Treasures of the Deep,PS,97.0,Action,Namco Bandai Games,0.10,0.07,0.00,0.01,0.18
906,Mini-Yonku Shining Scorpion: Let's & Go!!,SNES,96.0,Racing,ASCII Entertainment,0.00,0.00,0.90,0.00,0.90
2429,Casper,PS,96.0,Adventure,Interplay,0.46,0.31,0.00,0.05,0.82
3492,Glover,N64,98.0,Platform,Hasbro Interactive,0.72,0.17,0.00,0.01,0.90
4365,Adventure Island,NES,86.0,Platform,Hudson Soft,0.00,0.00,1.05,0.00,1.05


In [182]:
since2000 = (df['Year'] >= 0) & (df['Year'] < 30)
df[since2000].head(5)

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Total_Sales
74,Age of Empires III: Complete Collection,PC,9.0,Strategy,Microsoft Game Studios,0.01,0.12,0.0,0.03,0.16
213,NASCAR Unleashed,PS3,11.0,Racing,Activision,0.09,0.00,0.0,0.01,0.10
304,Kung Fu Panda: Showdown of Legendary Legends,PS3,15.0,Action,Little Orbit,0.00,0.02,0.0,0.00,0.02
768,NASCAR Thunder 2004,PS2,3.0,Racing,Electronic Arts,0.53,0.41,0.0,0.14,1.08
1015,Wolfenstein,PC,9.0,Shooter,Activision,0.01,0.03,0.0,0.01,0.05


In [183]:
def cleanYear(year):
  if year < 30:
    return year + 2000
  elif year < 100:
    return year + 1900
  else:
    return year

df['Year'] = df['Year'].apply(cleanYear)
df['Year'].fillna(df['Year'].mean(), inplace = True)  #결측치 평균값으로 처리
df['Year'] = df['Year'].apply(int)  #연도 int로 변경

since1900 = (df['Year'] < 100) & (df['Year'] >= 30)
since2000 = (df['Year'] >= 0) & (df['Year'] < 30)

print('1900년대 2자리수 연도 데이터 수:', len(df[since1900]))
print('2000년대 2자리수 연도 데이터 수:', len(df[since2000]))
print('년도 빈값:', df['Year'].isna().sum())


1900년대 2자리수 연도 데이터 수: 0
2000년대 2자리수 연도 데이터 수: 0
년도 빈값: 0


In [184]:
df.sort_values('Year', ascending=True, inplace=True)

In [185]:
df.dropna(inplace=True)

In [186]:
genres = df.Genre.value_counts()
genres.index

Index(['Action', 'Sports', 'Misc', 'Role-Playing', 'Shooter', 'Adventure',
       'Racing', 'Platform', 'Simulation', 'Fighting', 'Strategy', 'Puzzle'],
      dtype='object')

# 지역별 선호 게임 장르

In [187]:
def updateTitleLayout(f, title):
  f.update_layout(
    title=title,
    title_x=0.5,
    title_font_size = 25,
    font=dict(
        size=11,
        color="black"
    )
)

def getPreferenceGenre(area, start, end, top = 5):
  con = (df['Year'] >= start) & (df['Year'] <= end)
  data = df[con].groupby(['Genre'])[area].agg(**{'value':'sum'})
  data.sort_values(by='value', ascending=False, inplace=True)
  data.reset_index(inplace=True)
  return data[0:top]

def ShowPreferenceGenre(start, end):
  NA_Sales = getPreferenceGenre('NA_Sales', start, end)
  EU_Sales = getPreferenceGenre('EU_Sales', start, end)
  JP_Sales = getPreferenceGenre('JP_Sales', start, end)
  Other_Sales = getPreferenceGenre('Other_Sales', start, end)

  fig = make_subplots(rows = 2, cols = 2, subplot_titles=['NA','EU','JP','Other'], horizontal_spacing = 0.1, vertical_spacing=0.1)
  fig.add_bar (row = 1, col = 1, x = NA_Sales.Genre, y = NA_Sales.value)
  fig.add_bar (row = 1, col = 2, x = EU_Sales.Genre, y = EU_Sales.value)
  fig.add_bar (row = 2, col = 1, x = JP_Sales.Genre, y = JP_Sales.value)
  fig.add_bar (row = 2, col = 2, x = Other_Sales.Genre, y = Other_Sales.value)
  fig.update_layout(height = 800, width = 800, showlegend = False)
  fig.show()

ShowPreferenceGenre(1980, 2020)

# 연도별 게임 트렌드

In [188]:
def PreferenceByYear(area, target, start, end):
  con = (df['Year'] >= start) & (df['Year'] <= end)
  melt = pd.melt(df[con], id_vars=[target,'Year'], value_vars=area)
  melt_sum = melt.groupby(['Year', target])['value'].agg(**{'value':'sum'}).sort_values(by=['Year','value'], ascending=False)
  melt_sum.reset_index(inplace=True)
  return melt_sum

def showScatterByYear(data, target):
  fig = px.scatter(data, x = 'Year', y = 'value',
                color = target,
                hover_data=[target],
                height=800, width=1200).for_each_trace(lambda t: t.update(name=t.name.split("=")[1]))
  fig.update_layout(yaxis_title="출고량")
  for d in fig.data:
    d.update(mode='markers+lines')
  fig.show()

allArea = ['NA_Sales','EU_Sales','JP_Sales','Other_Sales']
data = PreferenceByYear('Total_Sales', 'Genre', 1980, 2020)
showScatterByYear(data, 'Genre')

# 출고량 분석 및 시각화

In [189]:
def getPreferenceGameByYear(area, start, end):
  con = (df_melt['Year'] >= start) & (df_melt['Year'] <= end) & (df_melt['variable'] == area)
  data = df_melt[con]
  data = data.groupby(['Year','Name','Genre']).sum()  #플랫폼 다른것 합침
  data.reset_index(inplace=True)
  return data

In [196]:
def showPreferenceGameByYear(area, title, start, end, top = 5):
  data = getPreferenceGameByYear(area, start, end)
  data = data.groupby(['Year']).apply(lambda grb: grb.nlargest(top, 'value'))
  data.reset_index(level=0, drop=True, inplace=True)
  data.reset_index(drop=True, inplace=True)

  fig = px.bar(data, x = 'Year', y = 'value',
                color = 'Genre',
                text = 'Name',
                height=800, width=1200, title=title).for_each_trace(lambda t: t.update(name=t.name.split("=")[1]))
  fig.update_layout(yaxis_title="출고량")
  updateTitleLayout(fig, title)
  fig.show()

df_melt = pd.melt(df, id_vars=['Name','Year','Genre','Platform'], value_vars=['NA_Sales','EU_Sales','JP_Sales','Other_Sales','Total_Sales'])
showPreferenceGameByYear('Total_Sales', '게임 순위 TOP 5', 1980, 2020)

In [197]:
showPreferenceGameByYear('NA_Sales', 'NA 게임 순위 TOP 5', 2010, 2016)
showPreferenceGameByYear('EU_Sales', 'EU 게임 순위 TOP 5', 2010, 2016)
showPreferenceGameByYear('JP_Sales', 'JP 게임 순위 TOP 5', 2010, 2016)
showPreferenceGameByYear('Other_Sales', 'Other 게임 순위 TOP 5', 2010, 2016)

In [198]:
df.head(5)

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Total_Sales
7317,Checkers,2600,1980,Misc,Atari,0.22,0.01,0.0,0.00,0.23
5938,Kaboom!,2600,1980,Misc,Activision,1.07,0.07,0.0,0.01,1.15
5723,Asteroids,2600,1980,Shooter,Atari,4.00,0.26,0.0,0.05,4.31
13133,Freeway,2600,1980,Action,Activision,0.32,0.02,0.0,0.00,0.34
14264,Boxing,2600,1980,Fighting,Activision,0.72,0.04,0.0,0.01,0.77


In [209]:
def showBarByYear(data, target):  
  fig = px.bar(data, x = 'Year', y = 'value',
                text = target,
                color = target,
                hover_data=[target],
                height=800, width=1200).for_each_trace(lambda t: t.update(name=t.name.split("=")[1]))
  fig.update_layout(yaxis_title="출고량")
  fig.show()

data = PreferenceByYear('Total_Sales', 'Platform', 2010, 2016)
showBarByYear(data, 'Platform')

In [201]:
def PreferenceByPublisher(area, target, start, end, top = 5):
  con = (df['Year'] >= start) & (df['Year'] <= end)
  melt = pd.melt(df[con], id_vars=[target,'Publisher'], value_vars=area)
  melt_sum = melt.groupby(['Publisher', target])['value'].agg(**{'value':'sum'}).sort_values(by=['value'], ascending=False)
  melt_sum.reset_index(inplace=True)
  publisher_sum = melt.groupby(['Publisher'])['value'].agg(**{'value':'sum'}).sort_values(by=['value'], ascending=False)
  publisher_sum.sort_values(by='value', ascending=False, inplace=True)
  publisher_sum.reset_index(inplace=True)
  topPublisherColumns = publisher_sum.head(top)['Publisher']
  melt_sum.set_index('Publisher', inplace=True)
  return melt_sum.loc[topPublisherColumns].reset_index()

def showBarByPublisher(data, target):  
  fig = px.bar(data, x = 'Publisher', y = 'value',
                text = target,
                color = target,
                hover_data=[target],
                height=800, width=1200).for_each_trace(lambda t: t.update(name=t.name.split("=")[1]))
  fig.update_layout(yaxis_title="출고량")
  fig.show()

data = PreferenceByPublisher('Total_Sales', 'Genre', 2010, 2020)
showBarByPublisher(data, 'Genre')

In [206]:
!pip install scipy --upgrade
from scipy.stats import chi2_contingency

In [212]:
def contingency(data1, data2):
  ct = pd.crosstab(data1, data2)
  chi2, pvalue, dof, data = chi2_contingency(ct, correction = False)
  print('chi2:',chi2, ' pvalue:', pvalue, 'dof:', dof)
  return pvalue

In [221]:
df_top50 = df.sort_values(by='Total_Sales', ascending=False).head(50)
pvalue1 = contingency(df_top50['Platform'], df_top50['Genre'])  #독립
pvalue2 = contingency(df_top50['Platform'], df_top50['Total_Sales'])  #독립
pvalue3 = contingency(df_top50['Genre'], df_top50['Total_Sales'])  #독립
pvalue4 = contingency(df_top50['Publisher'], df_top50['Total_Sales'])  #독립
pvalue5 = contingency(df_top50['Publisher'], df_top50['Genre']) #종속
pvalue6 = contingency(df_top50['Platform'], df_top50['Publisher'])  #종속

chi2: 128.72609599395315  pvalue: 0.10719205235929136 dof: 110
chi2: 542.2619047619048  pvalue: 0.3243889234816235 dof: 528
chi2: 493.92857142857144  pvalue: 0.3203451757903018 dof: 480
chi2: 196.09375000000003  pvalue: 0.40461831493238276 dof: 192
chi2: 108.0282738095238  pvalue: 3.6024395522240376e-08 dof: 40
chi2: 88.19940476190476  pvalue: 8.683971998024357e-05 dof: 44


In [222]:
px.bar(x=['Platform - Genre', 'Platform - Total_Sales', 'Genre - Total_Sales', 'Publisher - Total_Sales', 'Publisher - Genre', 'Platform - Publisher'],
       y = [pvalue1, pvalue2, pvalue3, pvalue4, pvalue5, pvalue6], height = 600, width = 800)

# 다음 분기에 어떤 게임을 설계해야 할까?

2010년도 부터 액션 슈팅 게임의 인기가 가장 높기 때문에
GTA나 콜오브 듀티같은 액션, 슈팅 게임을 만들어야 한다.
플랫폼은 PS,XBOX360,Wii로 출시하는 것이 좋다